In [146]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import seaborn as sns 
import matplotlib.colors

import time
from matplotlib.colors import LinearSegmentedColormap
pd.set_option('display.max_columns', None)


In [147]:
from nba_api.stats.static import teams

from nba_api.stats.endpoints import leaguedashplayerptshot, leaguedashplayerstats, synergyplaytypes

#### Get Touch Time DF

In [148]:
def get_touch_time_shots(season, touch_range):
    df = leaguedashplayerptshot.LeagueDashPlayerPtShot(season = season, touch_time_range_nullable = touch_range).get_data_frames()[0]
    
    
    df = df[['PLAYER_ID', 'PLAYER_LAST_TEAM_ID' , 'FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A']]
    
    shot_type = 'self_created'
    if touch_range == "Touch < 2 Seconds":
        shot_type = 'not_self_created'
        
    df['shot_type'] = shot_type
    return df

In [149]:
def get_touch_df(season):

    touch_range_list = ["Touch < 2 Seconds", "Touch 2-6 Seconds",  "Touch 6+ Seconds"]

    dataframes = []

    # Loop through touch_range_list and call the function for each value
    for touch_value in touch_range_list:
        print(touch_value)
        df = get_touch_time_shots(season = season, touch_range = touch_value)
        dataframes.append(df)
        time.sleep(3)
        
    touch_df = pd.concat(dataframes, ignore_index=True)

    touch_df = touch_df.groupby(['PLAYER_ID', 'PLAYER_LAST_TEAM_ID', 'shot_type']).agg(
        {**{col: 'first' for col in touch_df.columns if col not in ['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_LAST_TEAM_ID', 'shot_type']},
        **{col: 'sum' for col in touch_df.columns if col not in ['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_LAST_TEAM_ID', 'shot_type']}
        }
    ).reset_index()

    touch_df['efg'] = (touch_df.FGM + touch_df.FG3M*0.5)/touch_df.FGA

    suffix_mapping = {'not_self_created': '_not_self_created', 'self_created': '_self_created'}

    touch_df = touch_df.pivot_table(index=['PLAYER_ID', 'PLAYER_LAST_TEAM_ID' ], columns='shot_type', \
                                    values=['FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A', 'efg'], aggfunc='sum')

    # Flatten the MultiIndex columns
    touch_df.columns = [f'{col[0]}{suffix_mapping[col[1]]}' if col[1] in suffix_mapping else col[0] for col in touch_df.columns]

    touch_df = touch_df.reset_index()

    return(touch_df)

#### Get Synergy Data

In [150]:
def get_synergy_stats(season, play_type):

    synergy_df = synergyplaytypes.SynergyPlayTypes(season = season, play_type_nullable = play_type, per_mode_simple = 'Totals',\
                                      type_grouping_nullable = 'Offensive', player_or_team_abbreviation = 'P')\
                                .get_data_frames()[0]

    synergy_df = synergy_df[['PLAYER_ID', 'POSS', 'PTS']]


    synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()

    synergy_df['PPP'] = synergy_df.PTS/synergy_df.POSS

    for column in synergy_df.columns:
        # Check if the column name is not PLAYER_ID
        if column != 'PLAYER_ID':
            # Add the suffix based on the play_type variable
            new_column_name = column + "_" + play_type
            # Rename the column
            synergy_df.rename(columns={column: new_column_name}, inplace=True)

    return synergy_df

In [151]:
def get_synergy_df(season):

    synergy_df = poss_df[['PLAYER_ID']]

    play_types = ['Cut', 'Handoff', 'Isolation', 'OffScreen', 'Postup', 'PRBallHandler',\
                'PRRollman', 'Spotup', 'Transition']

    for play_type in play_types:
        print(play_type)
        synergy_df = pd.merge(synergy_df, get_synergy_stats(season, play_type),  how = 'left')
        time.sleep(3)

    return(synergy_df)
    

### Pull Data & Cleanse

In [152]:
season = '2024-25'

##### Pull Possession Data

In [153]:
poss_df = leaguedashplayerstats.LeagueDashPlayerStats(season=season, \
                                                  measure_type_detailed_defense='Advanced').get_data_frames()[0]



poss_df = poss_df[['PLAYER_ID', 'PLAYER_NAME', 'POSS', 'GP', 'MIN']]

poss_df = poss_df.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum().reset_index()

##### Pull Stats Data

In [154]:
print('\nPulling Touch Time Data')
touch_df = get_touch_df(season)

print('\nPulling Synergy Data')
synergy_df = get_synergy_df(season)


Pulling Touch Time Data
Touch < 2 Seconds
Touch 2-6 Seconds
Touch 6+ Seconds

Pulling Synergy Data
Cut
Handoff
Isolation
OffScreen
Postup
PRBallHandler
PRRollman
Spotup
Transition


##### Data Cleanse & Merge

In [155]:
off_ball_df = pd.merge(poss_df, touch_df, on = 'PLAYER_ID')

off_ball_df = pd.merge(off_ball_df, synergy_df, on = 'PLAYER_ID')


for column in off_ball_df.columns:
    if (column.startswith("FG") or column.startswith("PTS") or column.startswith("POSS_")):
        off_ball_df[column] = (off_ball_df[column] / off_ball_df['POSS']) * 75
        
#off_ball_df = off_ball_df[off_ball_df.POSS > 1000]

off_ball_df['headshot_url'] = "https://cdn.nba.com/headshots/nba/latest/1040x760/" + \
            off_ball_df.PLAYER_ID.astype(str) + \
                    ".png"

In [156]:
team_colors = pd.read_csv('Data/teamColors.csv')
team_colors.rename(columns={'TEAM_ID': 'PLAYER_LAST_TEAM_ID', 'Primary Color': 'primary_color'}, inplace=True)
team_colors = team_colors[['PLAYER_LAST_TEAM_ID', 'primary_color']]

off_ball_df = pd.merge(off_ball_df, team_colors, on = 'PLAYER_LAST_TEAM_ID')

##### Export Data

In [157]:
off_ball_df['season'] = season

off_ball_df.to_csv('Data/headshot_plot_obs_' + season + '.csv')

### Save Latest Update Time

In [158]:
from datetime import date

# Get today's date
today = date.today()

# Convert date to string
today_str = today.strftime("%Y-%m-%d")

# Define the file name
file_name = "latest_update.txt"

# Save the date as a text file
with open(file_name, "w") as file:
    file.write(today_str)

print(f"Today's date ({today_str}) saved in {file_name}")

Today's date (2024-12-24) saved in latest_update.txt
